# 1. Sequence fetching, Li et al.
___
Herschlag lab <br>
2020

In [3]:
from pathlib import Path
import time

import matplotlib.pyplot as pl
from Bio import SeqIO
from tqdm import tqdm
import pandas as pd
import numpy as np
import ast

import multiprocessing as mp
from subprocess import PIPE
import subprocess
import copy

## Description

### 1. Prepare the Li et al. organism list by pre-processing and chunking

In [2]:
liOrgListPath = 'Lietal_OrgTempList/temperature_data_full.tsv'

temped_Li_list = pd.read_csv(liOrgListPath, delimiter = '\t').drop_duplicates()
print(len(temped_Li_list), 'Unique Organisms')

21498 Unique Organisms


#### Split taxonomic information by genus and species

In [3]:
temped_Li_list['genus_name'] = temped_Li_list.organism.apply(lambda s: s.split('_')[0]).str.capitalize()
temped_Li_list['species_name'] = temped_Li_list.organism.apply(lambda s: s.split('_')[1])
temped_Li_list['genus_species'] = temped_Li_list.genus_name + ' ' + temped_Li_list.species_name

temped_Li_list.head(3)

,organism,domain,temperature,taxid,lineage_text,superkingdom,phylum,class,order,family,genus,genus_name,species_name,genus_species
0,abiotrophia_adiacens,Bacteria,37,46124,cellular organisms; Bacteria; Terrabacteria gr...,2,1239.0,91061.0,186826.0,186828.0,117563.0,Abiotrophia,adiacens,Abiotrophia adiacens
1,abiotrophia_balaenopterae,Bacteria,37,137733,cellular organisms; Bacteria; Terrabacteria gr...,2,1239.0,91061.0,186826.0,186828.0,117563.0,Abiotrophia,balaenopterae,Abiotrophia balaenopterae
2,abiotrophia_defectiva,Bacteria,37,46125,cellular organisms; Bacteria; Terrabacteria gr...,2,1239.0,91061.0,186826.0,186827.0,46123.0,Abiotrophia,defectiva,Abiotrophia defectiva


#### Remove any organisms without a defined species name (species_name == 'sp')

In [4]:
temped_Li_list_clean = temped_Li_list[['genus_species', 'temperature']].loc[temped_Li_list.species_name != 'sp']

display(temped_Li_list_clean.head(3))
print(len(temped_Li_list_clean), 'Clean Organisms')

,genus_species,temperature
0,Abiotrophia adiacens,37
1,Abiotrophia balaenopterae,37
2,Abiotrophia defectiva,37


20897 Clean Organisms


#### Fetching sequences will require chunking the organism list (to keep Entrez query lengths small enough)

In [5]:
CHUNKED_ROOT = Path('Lietal_OrgTempList/Chunked_Temp_List')
NUM_CHUNKS = 4

for index, df in enumerate(list(np.array_split(temped_Li_list_clean, NUM_CHUNKS))):
    chunkPath = CHUNKED_ROOT.joinpath('LiTempedOrgs_Chunk_{}.txt'.format(index+1))
    with open(chunkPath, 'w') as outPath:
        outPath.write('\n'.join(df.genus_species.values.tolist()))

In [6]:
# Combine the records for use in Entrezpy Fetching
!cat ./Lietal_OrgTempList/Chunked_Temp_List/*_Chunk_*.txt > ./Lietal_OrgTempList/Chunked_Temp_List/LiTempedOrgs_All.txt

#### Also export the full cleaned list for downstream use

In [7]:
temped_Li_list_clean.to_csv('Lietal_OrgTempList/LietAl_temperature_data_culled.csv')

<br>


## 3. Fetch the UIDs for each query

In [10]:
def enzymeListToDict(handle):
    """
    
    Arguments:
    
    Returns:
    
    
    """
    organisms = {}
    indices = []
    names = []
    index = 0
    with open(handle) as el:
        for line in el.readlines():
            if index%2 == 0:
                indices.append(line.strip()[1:])
            else:
                names.append(line.strip())
            index += 1
    names = dict(zip(indices, names))
    return names

def fetchOrganisms(filepath):
    """
    
    Arguments:
    
    Returns:
    
    """
    with open(filepath) as f:
        o = f.readlines()
    organisms = [x.strip() for x in o if not x.strip() == 'Escherichia coli' and not x.strip() == 'Pseudomonas aeruginosa']
    return organisms

In [23]:
def getUIDs(enzyme, outrecpath, chunkSize):
    """
    
    
    Arguments:
    
    Returns:
    
    """
    python = '/Users/varundeepakgudhe/opt/anaconda3/bin/python'
    program = 'Scripts/entrezpy_UIDfetch.py'
    email = 'vgudhe@ncsu.edu'
    orglistpath = 'Lietal_OrgTempList/Chunked_Temp_List/LiTempedOrgs_All.txt'
    
    def writeUIDsToDB(enzymeName, uidList, db_path):
        with open(db_path, "a+") as db:
            db.write('{}\t{}\t{}\n'.format(enzymeName, len(uidList), str(uidList)))

    uidList = []
    for i in range(4):
        args = [python, program, email, enzyme, i*chunkSize, (i+1)*chunkSize, orglistpath]
        statement = '{} {} -e {} -E "{}" -S {} -n {} -p "{}"'.format(*args)

        entrezQuery = subprocess.Popen(statement, stdout=PIPE, shell=True)
        results = entrezQuery.communicate()[0]
        n = ast.literal_eval(results.decode('utf-8').strip())
        uidList.extend(n)

    writeUIDsToDB(enzyme, uidList, db_path = outrecpath)

In [13]:
raw_enzyme_list = Path('BrendaEnzymeList/enzymeList_Brenda_Raw_biotin_carboxylase.txt')
raw_enzyme_series = pd.Series(enzymeListToDict(raw_enzyme_list))

# Remove missing entry, de-replicate
raw_enzyme_series_dereplicated = raw_enzyme_series.loc[~(raw_enzyme_series.values=='')].drop_duplicates()

print('{} enzymes in raw Brenda list\n{} unique enzymes after cleaning'.format(len(raw_enzyme_series), len(raw_enzyme_series_dereplicated)))

1 enzymes in raw Brenda list
1 unique enzymes after cleaning


#### a. Collect UIDs

In [14]:
# See 4a–b.
orglistpath = 'Lietal_OrgTempList/Chunked_Temp_List/LiTempedOrgs_All.txt'

enzymes = list(raw_enzyme_series_dereplicated.values)
orglist = fetchOrganisms(orglistpath)
chunkSize = int(len(orglist)/4)

##### Total time: ~7h

In [20]:
outrecpath = 'Lietal_FetchedSeqs/1912_Lietal_UIDLists.txt'

In [24]:
getUIDs("biotin carboxylase",outrecpath, chunkSize)

<br>


## 4. Parse GIs (UIDs for NR DB) and map to their accessions

#### a. Parse and dereplicate the UIDs (~5min)

In [26]:
uids = []
outrecpath = 'Lietal_FetchedSeqs/1912_Lietal_UIDLists.txt'

with open(outrecpath, 'r') as reader:
    l = reader.readline()
    while l:
        recuids = ast.literal_eval(l.split('\t')[-1])
        uids.extend(recuids)
        l = reader.readline()
        
# Dereplicate UIDs
uidSet = set(uids)

print('Total UID count pre-mapping: {}\nUnique UID count pre-mapping: {}'.format(len(uids),len(uidSet)))

Total UID count pre-mapping: 50265
Unique UID count pre-mapping: 37685


#### b. Write the dereplicated UIDs to a file

In [27]:
cleanUIDPath = 'Lietal_FetchedSeqs/CleanUIDsForMapping.txt'

with open(cleanUIDPath, 'a+') as uid_path:
    uid_path.write(''.join(['{}\n'.format(uid) for uid in uidSet]))

In [28]:
del(uids)
del(uidSet)

#### c. Perform the mapping from UID (GI for NR proteins) to Acc using the NCBI tool [gi2accession](https://ncbiinsights.ncbi.nlm.nih.gov/2016/12/23/converting-lots-of-gi-numbers-to-accession-version/) (~10min)

In [1]:
%%bash

cat Lietal_FetchedSeqs/CleanUIDsForMapping.txt | /Users/varundeepakgudhe/opt/anaconda3/envs/py27/bin/python Scripts/gi2accession.py -d /Volumes/MyPassport/Dr.Raffael/gi2acc_lmdb.db > Lietal_FetchedSeqs/MappedCleanUIDs.txt

#### d. Clean the mapped GIs->Accession

In [4]:
clean_UIDs = "Lietal_FetchedSeqs/MappedCleanUIDs.txt"
mappedAccessions = pd.read_csv(clean_UIDs, 
                               delimiter = '\t', 
                               names = ['GI', 'Acc', 'Length']
                               )

##### Identify the non-mapped GIs to be dropped

In [6]:
mappedAccessions['GI'] = mappedAccessions['GI'].astype(str)

In [7]:
missingAccessions = mappedAccessions.loc[(mappedAccessions.GI.str.contains('not found',na=False))]
missingAccessions

,GI,Acc,Length


In [8]:
mappedAccessionsCulled = mappedAccessions.drop(labels = missingAccessions.index.values)
mappedAccessionsCulled['GI'] = mappedAccessionsCulled.GI.astype(np.int32)
mappedAccessionsCulled['Acc'] = mappedAccessionsCulled.Acc.astype(str)
mappedAccessionsCulled['Length'] = mappedAccessionsCulled.Length.astype(np.int32)

print("{} of {} total accessions weren't successfully mapped ({}%)".format(len(missingAccessions), 
                                                                           len(mappedAccessions), 
                                                                           round(100*(len(missingAccessions)/len(mappedAccessions)), 
                                                                                 5)))
print("Proceeding with {} mapped and culled accessions".format(len(mappedAccessionsCulled)))

0 of 37685 total accessions weren't successfully mapped (0.0%)
Proceeding with 37685 mapped and culled accessions


In [9]:
# Examine a sample of the results!
mappedAccessionsCulled.iloc[0:3]

,GI,Acc,Length
0,-1662646231,CAK3272800.1,447
1,619865662,GAJ52212.1,1095
2,1456951037,SWJ88283.1,449


#### Hold the dictionary of Acc->GIs (a 1:1 mapping) in memory for part **5.**

In [10]:
accessionDict = mappedAccessionsCulled.set_index('Acc')['GI'].to_dict()

<br>

## 5. Parse the Non-Redundant Database (nr.gz) and extract a fasta of just our target sequences (1 pass)

In [11]:
import gzip
import copy
import re

from tqdm import tqdm
from Bio import SeqIO

### a. Prepare the maps needed to identify desired seqs in nr.gz and to gather the needed data in their new fasta headers
 - nr.gz has the protein Accession and the organism name in the headers. To do this we need to hold 2 dictionaries in memory while parsing:
     - Map from accession -> GI
     - Map from GI -> [all enzyme query terms returning that GI]
         - VERY IMPORTANT: A single GI can correspond to MORE THAN ONE ENZYME! (This alters how we parcel out the seqs and how we make the GI->Enzyme mapping below)

#### i. First read the list of GIs (as a string of strings) into memory

In [12]:
# Basic parser for the dumped UID lists to generate a flat dictionary to look up UID->Enzyme efficiently
UIDListoutrecpath = 'Lietal_FetchedSeqs/1912_Lietal_UIDLists.txt'

recs = {}
with open(UIDListoutrecpath, 'r') as outUIDs:
    l = outUIDs.readline()
    while l:
        data = l.strip().split('\t')[0:3]
        recs[data[0]] = data[2]
        l = outUIDs.readline()

#### ii. Then convert to a real list of strings

In [13]:
import ast

cleanrecs = {} # Enzyme -> unique set of GIs
for k, v in tqdm(recs.items()):
    cleanrecs[k] = set(ast.literal_eval(v)) # Just convert the recs to actual list. Dereplicate them BY ENZYME at the same time!

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  7.00it/s]


##### iii. Finally build the dictionary from GI->[EnzymeName1, EnzymeName2, ...]
 - VERY IMPORTANT: A single GI can correspond to MORE THAN ONE ENZYME! (This alters how we parcel out the seqs and how we make the GI->Enzyme mapping in this cell)
 - We also index prepend a padded 10-digit index in front of the enzyme name (`str(index).zfill(10)+enzymeName`), which will be parsed out later

In [14]:
# VERY IMPORTANT: A single GI can correspond to MORE THAN ONE ENZYME! (This alters how we parcel out the seqs and how we make the GI->Enzyme mapping in this cell)
# Goal:bBuild a flat dictionary from GI:[EnzymeName1+'+{}'.format(index), EnzymeName2+'+{}'.format(index), ...]

flatDB = {}
already_present_count = 0
for enzymeName, GISet in tqdm(cleanrecs.items()):
    for index, GI in enumerate(GISet): # For a given enzyme & associated list of GIs
        # Catch cases where one GI corresponds to more than one enzyme!
        if GI in flatDB.keys():
            flatDB[GI].append(str(index).zfill(10)+enzymeName)
            already_present_count+=1
        else:
            flatDB[GI] = [str(index).zfill(10)+enzymeName]

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 25.58it/s]


#### iv. Dump the resulting dictionary to a JSON file in case we have to restart the kernel (as generating it was time intensive)


In [16]:
import json

with open('Lietal_FetchedSeqs/GI_to_Enzyme_Map.json', 'a+') as fp:
    json.dump(flatDB, fp)

##### And just for curiosity's sake...

In [17]:
moreThan1inflatDB = {k:v for k, v in flatDB.items() if len(v) > 1}

print('# GIs dereplicated on enzyme AND across enzymes: {}'.format(len(flatDB)))
print('# GIs appearing in more than 1 enzyme: {}'.format(len(moreThan1inflatDB)))

# GIs dereplicated on enzyme AND across enzymes: 37685
# GIs appearing in more than 1 enzyme: 0


##### Free the intermediate dictionaries to free up memory

In [18]:
del(recs)
del(cleanrecs)

### b. Parse nr.gz entry by entry, identify seqs corresponding to GIs we want to keep, and write each desired seq to a fasta file with the corresponding query term(s) with a custom header (~3h)

In [19]:
nrgs = '/Volumes/MyPassport/Dr.Raffael/nr.gz'
parsedEnzymeRoot = Path('Lietal_FetchedSeqs/parsed_fastas')

#### Dictionaries
# accessionDict
# flatDB->Enzyme(s)

with gzip.open(nrgs, 'rt') as handle:
    for record in tqdm(SeqIO.parse(handle, 'fasta'), total = 595907626, desc = 'Matching Seqs'):
        # '\x01' invisible delimiter
        splitHeaders = record.description.split('\x01')
        acc_org = []
        for entry in splitHeaders:
            acc = entry.split(' ')[0]
            m = re.search('\[.*?\]$', entry)
            if m:
                organism = m.group()[1:-1]
            else:
                organism = 'Missing'
            acc_org.append([acc, organism])
            
        for a, org in acc_org:
            if a in accessionDict.keys():
            
                gi = str(abs(accessionDict[a]))
             
                try:
                    for enzyme in [e[10:] for e in flatDB[gi]]:
                        r = copy.copy(record)
                        r.id = '{}|{}|{}|{}'.format(a, gi, enzyme, org)
                        r.name = ''
                        r.description = ''
                        outFastaPath = parsedEnzymeRoot.joinpath(Path('{}.fa'.format(enzyme.replace(' ', '_').replace('/', '__'))))
                        with open(outFastaPath, 'a+') as h:
                            SeqIO.write([r], h, 'fasta')
                except:
                    pass
            else:
                pass

Matching Seqs: 100%|█████████| 595907626/595907626 [2:07:31<00:00, 77879.14it/s]


### c. Characterize the written files to check how the parsing went
    - How many files were written?
    - How many sequences do each file have in them?
    - What's the distribution of the lengths of sequences in each file?
    - What fraction of sequences in each file have a missing organism or organism without a species identifier?

In [20]:
names_sizes = []
for path in tqdm(parsedEnzymeRoot.iterdir()):
    if path.is_file():
        recs = [record for record in SeqIO.parse(path, 'fasta')]
        lens = [len(s) for s in recs]
        unique_organisms = set([r.description.split('|')[-1] for r in recs if 'Missing' not in r.description])

        names_sizes.append({'Name': Path(path).stem, 
                            'File_Size': Path(path).stat().st_size,
                            'Num_Seqs': len(recs),
                            'Mean_Len': np.mean(lens),
                            'Std_Len': np.std(lens),
                            'Num_Unique_Organisms': len(unique_organisms)
                           })

names_sizes_df = pd.DataFrame(names_sizes)

1it [00:00, 14.12it/s]


In [21]:
names_sizes_df.sort_values('Num_Unique_Organisms', ascending = False)

,Name,File_Size,Num_Seqs,Mean_Len,Std_Len,Num_Unique_Organisms
0,biotin_carboxylase,4294006,8137,451.398304,171.93907,3132


## 6. Generate a cleaned copy of the fastas with seqs of organisms in our final bacterial tree only, and seqs consisting of only canonical residues, in the proper *Genus species* form (remove sub-species information)
    - Remove sequences corresponding to organisms that do not appear in the culled tree
    - Remove sequences containing non-standard residues
    - Clean the fasta headers such that the organism header is solely 'Genus species'

### a. Import the list of organisms from the final bacterial tree. This is our "reference" set

In [22]:
from Bio import Phylo

treeRoot = Path('/Users/varundeepakgudhe/Downloads/Dr.Rafael_prev_res_notebooks/Tree_Generation/Processed_Trees/Final_BacterialTrees')
tree = Phylo.read(treeRoot.joinpath('bac120_r86.2_dedup_droppedTipsRenamed_genusSpecies_nonCollapsed.phyloxml'), 'phyloxml')
treeOrganisms = {t.name for t in tree.get_terminals()}

print('The tree has {} organisms'.format(len(treeOrganisms)))

The tree has 5852 organisms


### b. Our allowed amino acids

In [23]:
#from Bio.Alphabet.IUPAC import IUPACProtein

PROT_LETTERS = {'E', 'C', 'N', 'R', 'A', 'M', 'P', 'H', 'D', 'V', 'I', 'Y', 'K', 'F', 'W', 'T', 'G', 'Q', 'L', 'S'}
print('Protein Alphabet: {}'.format(PROT_LETTERS))

Protein Alphabet: {'W', 'L', 'I', 'D', 'V', 'G', 'T', 'F', 'P', 'C', 'K', 'Y', 'R', 'E', 'M', 'N', 'A', 'Q', 'H', 'S'}


### c. Execute the cleaning (~45 min)

In [26]:
oldRoot = Path('Lietal_FetchedSeqs/parsed_fastas/')

newRoot = Path('Lietal_FetchedSeqs/parsed_fastas_cleaned')


cleaning_summary = []
for oldFile in tqdm(oldRoot.iterdir()):
    if oldFile.is_file() and ".ipynb_checkpoints" not in oldFile.parts:
        newFile = newRoot.joinpath(oldFile.name)
        culledCount = 0
        retainedCount = 0
        escapingCondition = 0
        escapees = []
        with open(newFile, 'a+') as nf:
            for record in SeqIO.parse(oldFile, 'fasta'):
                organism = record.description.split('|')[-1]
                orglist = organism.split(' ')
             
                # Want to short circuit with non-costly crit first to save time
                if organism == 'Missing': #No organism
                    culledCount+=1
                    pass
                elif len(orglist) == 1: #Catched genus but no species
                    culledCount+=1
                    pass
                elif not all([l in PROT_LETTERS for l in set(str(record.seq))]):
                    culledCount+=1
                    pass # Check that only IUPAC residues in the sequence
                else:     
                    r = copy.copy(record)
                    r.id = '|'.join([*record.description.split('|')[0:3], ' '.join(orglist[0:2])])
                    r.name = ''
                    r.description = ''
                    SeqIO.write(r, nf, 'fasta')
                    retainedCount+=1
                 
        try: 
            esccount = pd.Series([' '.join(r[0:2]) in treeOrganisms for r in escapees]).value_counts().loc[True] 
        except:
            esccount = 0 

        cleaning_summary.append({'Enzyme': oldFile.stem, 'Culled': culledCount, 'Retained': retainedCount, 'EscapingConditions': escapingCondition, 'Escapees_In_Tree': esccount})
        # Note that 'Escapees_In_Tree' should all be 0. If they're not, we need to figure out why and catch that condition
        # Also note that 'Enzyme' is the file handle, so EnzymeName.replace(' ', '_').replace('/', '__')

cleaning_summary_df = pd.DataFrame(cleaning_summary)
display(cleaning_summary_df.head())

0it [00:00, ?it/s]/var/folders/lk/f1snn09n3svd7pdwh0yw2pp40000gn/T/ipykernel_2446/1936245981.py:52: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  esccount = pd.Series([' '.join(r[0:2]) in treeOrganisms for r in escapees]).value_counts().loc[True]
1it [00:00,  4.61it/s]


,Enzyme,Culled,Retained,EscapingConditions,Escapees_In_Tree
0,biotin_carboxylase,44,8093,0,0


### d. Sanity checks—absolutely essential
1. Do all of the uncaught condition records in **c.** correspond to sequences with organisms not in the tree?
2. Do the number of fastas match the number of non-zero counts of expected UIDs (approximately)?
    - nr.gz parsing will generate a *new* fasta only if at least one instance of that enzyme's GIs exists in it
    - Thus, we expect the number of total enzymes with ≥1 UID to *exceed* the number of enzyme fastas written from parsing nr.gz since some "singletons" won't be either mapped to accessions or found in the frozen nr.gz database. This should be only a small fraction of all the enzymes though, and should only apply to enzymes with low returned UID counts
3. Do the number of Culled + Retained closely match (they won't perfectly due to imperfect mapping) the UID count fetched originally? It's important to consider the DEREPLICATED UID count, since UIDs would be fetched only once (They are dereplicated in **4.**).  This is a good general check...

##### Good news: the escaping seqs weren't in the tree, as expected

In [27]:
# 1.
cleaning_summary_df.Escapees_In_Tree.value_counts()

0    1
Name: Escapees_In_Tree, dtype: int64

##### Parse the UID list again to generate a record (~10min)

In [28]:
uid_recs = []
outrecpath = 'Lietal_FetchedSeqs/1912_Lietal_UIDLists.txt'

with open(outrecpath, 'r') as reader:
    l = reader.readline()
    while l:
        splitrec = l.split('\t')
        recuids = ast.literal_eval(splitrec[-1])
        derep_recuids = set(recuids)
        name = splitrec[0]
        count = splitrec[1]
        uid_recs.append({'Enzyme': name, 'Expected_Count': int(count),'Raw_UID_Count': len(recuids), 'Derep_UID_Count': len(derep_recuids)})
        l = reader.readline()

uid_rec_df = pd.DataFrame(uid_recs)

In [29]:
uid_rec_df['Enzyme'] = uid_rec_df.Enzyme.str.replace(' ', '_').str.replace('/', '__')
uid_rec_df.sort_values('Expected_Count', ascending = False, inplace = True)
uid_rec_df.set_index('Enzyme', inplace = True)
display(uid_rec_df.head())

# Another little sanity check. We really did fetch every single UID expected from every query.
print('Number of missing UIDs -> Enzymes')
(uid_rec_df.Expected_Count - uid_rec_df.Raw_UID_Count).value_counts()

,Expected_Count,Raw_UID_Count,Derep_UID_Count
Enzyme,,,
biotin_carboxylase,50265,50265,37685


Number of missing UIDs -> Enzymes


0    1
dtype: int64

#### NOTE: Again, we expect Derep_UID_Count to EXCEED the number of UIDs actually present in our GI->Enzyme mapper because there are redundancies in GI *across enzymes* not accounted for here. About 5,000,000 (<10%) in fact.
    - The plots show that we collected almost every sequence for every single enzyme. 
    - We were only truly missing 76471 sequences, or ~0.1% of all sequences.

In [31]:
joinedFetchedAndExpected = uid_rec_df.join(cleaning_summary_df.set_index('Enzyme'))
joinedFetchedAndExpected['Total_Seqs_Gathered'] = joinedFetchedAndExpected.Culled + joinedFetchedAndExpected.Retained
joinedFetchedAndExpected['Percent_Expected_Gathered'] = 100*(joinedFetchedAndExpected.Total_Seqs_Gathered)/(joinedFetchedAndExpected.Derep_UID_Count)
joinedFetchedAndExpected['Retained_Percent_ofGathered'] = 100*(joinedFetchedAndExpected.Retained)/(joinedFetchedAndExpected.Total_Seqs_Gathered)

display(joinedFetchedAndExpected.head())

,Expected_Count,Raw_UID_Count,Derep_UID_Count,Culled,Retained,EscapingConditions,Escapees_In_Tree,Total_Seqs_Gathered,Percent_Expected_Gathered,Retained_Percent_ofGathered
Enzyme,,,,,,,,,,
biotin_carboxylase,50265,50265,37685,44,8093,0,0,8137,21.592145,99.45926


#### This is how many sequences are truly "unaccounted for" (Presumably they weren't present in nr.gz, since only a couple hundred seqs were not successfully mapped from GI -> Acc).
    - Alternative possibility was that there were malformed fastas or errors in the databases that resulted in "false negatives"

In [32]:
print(int((joinedFetchedAndExpected.Derep_UID_Count*((100-joinedFetchedAndExpected.Percent_Expected_Gathered)/100)).sum()), "Missing")

29548 Missing


#### Write these statistics to a file as a record

In [33]:
joinedFetchedAndExpected.to_csv('Lietal_FetchedSeqs/parsed_fastas_cleaned_stats.csv')

#### Which enzymes had a low percent of expected seqs obtained?

In [34]:
joinedFetchedAndExpected.loc[joinedFetchedAndExpected.Percent_Expected_Gathered < 90].sort_values('Percent_Expected_Gathered').head(50)

,Expected_Count,Raw_UID_Count,Derep_UID_Count,Culled,Retained,EscapingConditions,Escapees_In_Tree,Total_Seqs_Gathered,Percent_Expected_Gathered,Retained_Percent_ofGathered
Enzyme,,,,,,,,,,
biotin_carboxylase,50265,50265,37685,44,8093,0,0,8137,21.592145,99.45926


In [35]:
joinedFetchedAndExpected.sort_values('Total_Seqs_Gathered', ascending = False)

,Expected_Count,Raw_UID_Count,Derep_UID_Count,Culled,Retained,EscapingConditions,Escapees_In_Tree,Total_Seqs_Gathered,Percent_Expected_Gathered,Retained_Percent_ofGathered
Enzyme,,,,,,,,,,
biotin_carboxylase,50265,50265,37685,44,8093,0,0,8137,21.592145,99.45926


## 7. Finally, generate a set of with the organismal growth temperatures added
- These fastas have many replicates of the SAME sequence belonging to the same organism and different organisms (up to 75% or more for some!)
    - BUT we can't and shouldn't dereplicates these sequences now as we need to construct a consensus sequence on a per-organism level AFTER we've done the alignment
- And although we restricted our queries to NOT include P. aeruginosa and E. coli, many such sequences got returned
    - But we're not going to worry about filtering them out here, because for the biggest returned sets of seqs, they're not dominating the counts

### a. Grab the Li temped list

In [36]:
liOrgListPath = 'Lietal_OrgTempList/temperature_data_full.tsv'

temped_Li_list = pd.read_csv(liOrgListPath, delimiter = '\t').drop_duplicates()
temped_Li_list['genus_name'] = temped_Li_list.organism.apply(lambda s: s.split('_')[0]).str.capitalize()
temped_Li_list['species_name'] = temped_Li_list.organism.apply(lambda s: s.split('_')[1])
temped_Li_list['genus_species'] = temped_Li_list.genus_name + ' ' + temped_Li_list.species_name

organism_temp_dict = temped_Li_list[['genus_species', 'temperature']].set_index('genus_species').squeeze().to_dict()

### b. Append the organismal growth temperatures from Li et al. to the fasta headers and re-write seqs

In [42]:
oldRoot = Path('Lietal_FetchedSeqs/parsed_fastas_cleaned')

newOut = Path('Lietal_FetchedSeqs/parsed_fastas_cleaned_temped')

for parsingFile in tqdm(oldRoot.iterdir()):
    if parsingFile.is_file() and ".ipynb_checkpoints" not in parsingFile.parts:
        newFile = newOut.joinpath(parsingFile.name)
        with open(newFile, 'a+') as nf:
            recs = []
            inlist_count=0
            extra_count=0
            for record in SeqIO.parse(parsingFile, 'fasta'):
                try:
                    r = copy.copy(record)
                    organism = r.description.split('|')[-1]
                    r.description = r.description+'|{}'.format(organism_temp_dict[organism])
                    recs.append(r)
                
                    inlist_count+=1
                except:
                    extra_count+=1
                    
            SeqIO.write(recs, nf, 'fasta')
print("Count of organisms fastas which are in our org list:", inlist_count,"\nCount of organisms fastas which are not in our org list:",extra_count)

1it [00:00,  5.58it/s]

Lietal_FetchedSeqs/parsed_fastas_cleaned/biotin_carboxylase.fa
Count of organisms fastas which are in our org list: 7225 
Count of organisms fastas which are not in our org list: 868


## MAFFT

In [45]:
import subprocess
from Bio import AlignIO

def run_mafft_large_dataset(input_fasta_file, output_aligned_file):
    # Command to run MAFFT with auto parameter selection
    # The --thread -1 option tells MAFFT to use all available CPU cores for the alignment, which can significantly speed up the process for large datasets.
    mafft_command = ["mafft", "--auto", "--thread", "-1", input_fasta_file]
    
    # Execute MAFFT and capture the output
    with open(output_aligned_file, 'w') as outfile:
        subprocess.run(mafft_command, stdout=outfile, check=True)
    
    # Load and print the first few alignments to verify (optional step)
    try:
        alignment = AlignIO.read(output_aligned_file, "fasta")
        print("Alignment of first 5 sequences:")
        for record in alignment[:5]:  # Print only the first 5 alignments
            print(record.id)
    except Exception as e:
        print(f"Error reading the alignment output: {e}")

input_fasta = "Lietal_FetchedSeqs/parsed_fastas_cleaned_temped/biotin_carboxylase.fa"
output_aligned = "Lietal_FetchedSeqs/MAFFT/biotin_carboxylase.fa"

run_mafft_large_dataset(input_fasta, output_aligned)


OS = darwin
The number of physical cores =  8
nthread = 8
nthreadpair = 8
nthreadtb = 8
ppenalty_ex = 0
stacksize: 8176 kb
rescale = 1
Gap Penalty = -1.53, +0.00, +0.00



Making a distance matrix ..
 7201 / 7225 (thread    5)
done.

Constructing a UPGMA tree (efffree=0) ... 
 7220 / 7225
done.

Progressive alignment 1/2... 
STEP  6401 / 7224 (thread    5)
Reallocating..done. *alloclen = 6033
STEP  7001 / 7224 (thread    1)
Reallocating..done. *alloclen = 8469

Reallocating..done. *alloclen = 9741
STEP  7201 / 7224 (thread    6) h
Reallocating..done. *alloclen = 10932

done.

Making a distance matrix from msa.. 
 7200 / 7225 (thread    0)
done.

Constructing a UPGMA tree (efffree=1) ... 
 7220 / 7225
done.

Progressive alignment 2/2... 
STEP  6901 / 7224 (thread    0) h
Reallocating..done. *alloclen = 6107
STEP  7101 / 7224 (thread    5) h
Reallocating..done. *alloclen = 7943

Reallocating..done. *alloclen = 10256
STEP  7201 / 7224 (thread    2) h
done.

disttbfast (aa) Version 7.520
a

Alignment of first 5 sequences:
OIQ08369.1|1101268428|biotin
OAB45617.1|1025868728|biotin
EOD69354.1|486087350|biotin
CAG9202857.1|2094294216|biotin
PPI56038.1|1345285717|biotin


___